<a href="https://colab.research.google.com/github/astrid12345/recyclo/blob/convert_taco_to_yolo/scripts/convert_taco_dataset_to_yolo_format.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of this notebook is to convert a COCO-ish dataset to a format that can be used to train an Ultralytics YOLO model.

# Using this notebook: workflow

To use, make a copy of this notebook, and adapt it to work with your specific dataset. Please save your version of this ipynb file on GitHub in *recyclo/scripts*.

(File > Save a copy in GitHub > File path = "scripts/my_filename.ipynb" to save notebook in scripts folder)

Once you've generated your YOLO dataset, and are confident you can train a model with it, please upload your converted dataset to the Recyclo datasets google drive, https://drive.google.com/drive/folders/1bUkIYQRXX08OKI5TuOSg-eqntSudGaFB.

(Why Google Drive? Because these datasets are too large for GitHub!)

# What's in this notebook: contents

Notebook contents:
- intro to YOLO
- intro to COCO
- dataset specific notes (update for your specific dataset)
- convert dataset to generic COCO (update for your specific dataset)
- convert generic COCO to YOLO

# Pro tips about Colabs

When you open the "Files" tab on the left, you'll find yourself in a folder containing
* ..
* sample data

This is a colab thing, the "content" folder, to get you started.
Ignore it: click the .. to go up a level.

# Intro to YOLO

## General

In general, YOLO models output the following for a given image:
* Bounding box
* Class label
* Confidence score

To train a YOLO model, we need object detection datasets that contain images of what we're looking for (trash), and annotations: class labels and bounding boxes.

## Ultralytics YOLO

In this project we will use Ultralytics YOLO object detection, eg their YOLO11n model. YOLO11n is a pretrained object detection model developed by Ultralytics.

YOLO expects datasets in the following format:

```
dataset/
├── images/
│   ├── train/  <-- image files for training.
│   ├── val/    <-- image files for validation after each epoch. Must not overlap with images in train.
|   └── test/   <-- optional: can put some image files here for benchmarking.
├── labels/
│   ├── train/  <-- one .txt file per train image (must have same name). Contains class and bbox info..
│   ├── val/    <-- one .txt file per val image.
|   └── test/   <-- one .txt file per test image.
└── data.yaml   <-- config file; helps tie all the above together.
```

Example labels/train file:
```
<class_id> <x_center> <y_center> <width> <height>
```

Example data.yaml file:
```
path: /content/dataset  # Root folder
train: images/train
val: images/val

nc: 5  # number of classes
names: ['bottle', 'can', 'plastic bag', 'wrapper', 'paper']

```

# Intro to COCO

## General
COCO, Common Objects in Context, is a object detection, segmentation, and captioning dataset developed by Microsoft. It uses an annotations.json file to organize image data. This json annotation approach has become standard for other datasets to use.

Lots of datasets use COCO-style of formatting. In addition to the training images themselves, these datasets have at least one annotations json file which contains the following:
*   "images":  List of image metadata
*   "annotations":  List of label data (type of trash, bounding box definition, segmentation data; corresponds to images list)
*   "categories":  List of the different categories this dataset uses

An example of a COCO-style dataset file structure is as follows:
```
dataset/
├── annotations/
│   ├── instances_train2017.json
│   ├── instances_val2017.json
│   ├── person_keypoints_train2017.json
│   ├── captions_train2017.json
│   └── ... (other task-specific .json files)
├── images/
│   ├── train2017/
│   │   ├── 000000000009.jpg
│   │   ├── 000000000025.jpg
│   │   └── ...
│   └── val2017/
│       ├── 000000000139.jpg
│       ├── 000000000285.jpg
│       └── ...
└── LICENSE.txt (optional)
```

## COCO format required by the COCO-to-YOLO conversion function


To use the COCO to YOLO conversion function below, your data set must conform to the following (vastly simplified) COCO-like directory structure and json structure. It's unlikely that your dataset will conform to these specifications out of the box, so please use the code section below to modify your data's structure to match.

The COCO directory structure must be as follows, with a folder called "dataset" located in your "content" directory:
```
dataset/
├── images/
│   ├── 000001.jpg  # or png or whatever
│   ├── 000002.jpg
│   └── ...
└── annotations.json
```

And the annotations.json file must contain information in the following structure, and using the following json keywords:
```
{
  "images": [
    {
      "id": 1,
      "file_name": "000001.jpg",
      "width": 640,
      "height": 480
    },
    {
      "id": 2,
      "file_name": "000002.jpg",
      "width": 800,
      "height": 600
    }
  ],
  "annotations": [
    {
      "id": 1,
      "image_id": 1,
      "category_id": 1,
      "bbox": [100, 120, 50, 60],
      "area": 3000,
      "iscrowd": 0
    },
    {
      "id": 2,
      "image_id": 2,
      "category_id": 2,
      "bbox": [20, 30, 40, 50],
      "area": 2000,
      "iscrowd": 0
    }
  ],
  "categories": [
    {
      "id": 1,
      "name": "plastic"
    },
    {
      "id": 2,
      "name": "metal"
    }
  ]
}
```


# Helpful functions

This section has some helpful functions you can use later in this notebook.

In [1]:
import json
from pprint import pprint

def show_first_two_per_category(json_path):
    """
    Prints the first two entries of each root-level list in a JSON file.

    Useful for quickly inspecting the structure and content of an COCO-style
    annotations json file.

    It pretty-prints the first two entries of each top-level key that contains a list.

    Args:
        json_path (str or Path): Path to the JSON file to inspect.

    Raises:
        FileNotFoundError: If the provided path does not point to an existing file.
        json.JSONDecodeError: If the file is not valid JSON.
    """
    json_path = Path(json_path)

    if not json_path.exists():
        print(f"File not found: {json_path}")
        return

    with open(json_path, 'r') as f:
        data = json.load(f)

    for key, value in data.items():
        print(f"\n--- {key.upper()} (showing first 2 entries) ---")
        if isinstance(value, list):
            for item in value[:2]:
                pprint(item)
        else:
            print(f"{key} is not a list, skipping.")


---
⚠️ ***CHANGE THIS FILE FROM HERE DOWN TO SUIT YOUR DATASET*** ⚠️

The sections above apply for all dataset conversions.

---

In [5]:
# Sometimes this has to be run twice for the data to show up in the file tree
import kagglehub
from pathlib import Path

taco_dataset_path = Path(kagglehub.dataset_download('kneroma/tacotrashdataset'))  # https://www.kaggle.com/datasets/kneroma/tacotrashdataset
print(f"Dataset downloaded to {taco_dataset_path}\n")

Dataset downloaded to /kaggle/input/tacotrashdataset



In [14]:
# If your dataset has one or more annotations json files, inspect them to help discover some of the quirks your dataset has.
show_first_two_per_category(taco_dataset_path / "data" / "annotations.json")


--- INFO (showing first 2 entries) ---
info is not a list, skipping.

--- IMAGES (showing first 2 entries) ---
{'coco_url': None,
 'date_captured': None,
 'file_name': 'batch_1/000006.jpg',
 'flickr_640_url': 'https://farm66.staticflickr.com/65535/33978196618_632623b4fc_z.jpg',
 'flickr_url': 'https://farm66.staticflickr.com/65535/33978196618_e30a59e0a8_o.png',
 'height': 2049,
 'id': 0,
 'license': None,
 'width': 1537}
{'coco_url': None,
 'date_captured': None,
 'file_name': 'batch_1/000008.jpg',
 'flickr_640_url': 'https://farm66.staticflickr.com/65535/47803331152_19beae025a_z.jpg',
 'flickr_url': 'https://farm66.staticflickr.com/65535/47803331152_ee00755a2e_o.png',
 'height': 2049,
 'id': 1,
 'license': None,
 'width': 1537}

--- ANNOTATIONS (showing first 2 entries) ---
{'area': 403954.0,
 'bbox': [517.0, 127.0, 447.0, 1322.0],
 'category_id': 6,
 'id': 1,
 'image_id': 0,
 'iscrowd': 0,
 'segmentation': [[561.0,
                   1238.0,
                   568.0,
               

# TACO dataset

The TACO dataset uses COCO-style formatting (segmentation).

## Quirks

TACO has 15 batch folders, each containing jpgs with names like 000000.jpg, 000001.jpg, 000002.jpg, etc.
* Some numbers are skipped, eg 000000.jpg, 000001.jpg, 000003.jpg.
* The name 000000.jpg is used in multiple batch folders, to name different image files.
* It contains segmentation data (will be discarded).
* It uses categories (specific) and supercategories (more general) - only the supercategories will be retained.
* It includes scene categories (eg clearn streets) - this will be discarded.
  * It might be interesting in the future to consider using a multi-tasking model - detecting trash and also classsifying the scene might improve the models performance by adding a bit of context awareness.

## Conversion
To convert the TACO dataset to a format ultralytics YOLO can use, we must:
* Give the images unique names
* Split the TACO images into train and val sets
* Extract label and bbox info from annotations.json, and save it in individual txt files corresponding to the image files
* Make a data.yaml file

In [12]:
# Modify your dataset structure to match the required simple-COCO style
import shutil

source_root = taco_dataset_path / "data"
annotations_path = source_root / 'annotations.json'

target_root = Path('/content/dataset')
target_img_dir = target_root / 'images'
if target_root.exists():
    shutil.rmtree(target_root)
target_img_dir.mkdir(parents=True, exist_ok=True)

with annotations_path.open('r') as f:
    annotations = json.load(f)

# Reassign file names with unique zero-padded numbering
image_id_map = {}
for idx, image_info in enumerate(annotations['images']):
    old_path = source_root / image_info['file_name']
    new_filename = f"{idx:06}.jpg"
    new_path = target_img_dir / new_filename

    image_id_map[old_path] = new_path
    image_info['file_name'] = new_filename

    # Keep only required image fields
    annotations['images'][idx] = {
        'id': image_info['id'],
        'file_name': new_filename,
        'width': image_info['width'],
        'height': image_info['height']
    }

# Prune annotation fields
for ann in annotations['annotations']:
    ann_keys = ['id', 'image_id', 'category_id', 'bbox', 'area', 'iscrowd']
    for key in list(ann.keys()):
        if key not in ann_keys:
            del ann[key]

# Update category entries to only include id and name = supercategory
for cat in annotations['categories']:
    cat_id = cat['id']
    cat_super = cat.get('supercategory', f'category_{cat_id}')
    cat.clear()
    cat['id'] = cat_id
    cat['name'] = cat_super

# Copy image files to new location
for src, dst in image_id_map.items():
    if src.exists():
        shutil.copy(src, dst)
    else:
        print(f"Warning: Missing image: {src}")

# Remove unwanted top-level keys
for key in ['info', 'scene_annotations', 'scene_categories', 'licenses']:
    annotations.pop(key, None)

# Save cleaned annotations
new_annotations_path = target_root / 'annotations.json'
with new_annotations_path.open('w') as f:
    json.dump(annotations, f)

print(f"Converted and cleaned {len(image_id_map)} images.")

Converted and cleaned 1500 images.


In [13]:
# Inspect your new annotations.json file to make sure the format and field names match the simple COCO format described above
show_first_two_per_category(target_root / "annotations.json")


--- IMAGES (showing first 2 entries) ---
{'file_name': '000000.jpg', 'height': 2049, 'id': 0, 'width': 1537}
{'file_name': '000001.jpg', 'height': 2049, 'id': 1, 'width': 1537}

--- ANNOTATIONS (showing first 2 entries) ---
{'area': 403954.0,
 'bbox': [517.0, 127.0, 447.0, 1322.0],
 'category_id': 6,
 'id': 1,
 'image_id': 0,
 'iscrowd': 0}
{'area': 1071259.5,
 'bbox': [1.0, 457.0, 1429.0, 1519.0],
 'category_id': 18,
 'id': 2,
 'image_id': 1,
 'iscrowd': 0}

--- CATEGORIES (showing first 2 entries) ---
{'id': 0, 'name': 'Aluminium foil'}
{'id': 1, 'name': 'Battery'}


In [ ]:
import json
import shutil
import random
from pathlib import Path
from sklearn.model_selection import train_test_split
import yaml

# === CONFIG ===
input_root = Path(taco_dataset_path)
output_root = Path('/kaggle/working/taco_yolo')   # Since TACO is a kaggle dataset, this will output to kaggle/working/taco_yolo
train_ratio = 0.8  # 80% training, 20% validation

# === OUTPUT STRUCTURE ===
images_train = output_root / 'images' / 'train'
images_val = output_root / 'images' / 'val'
labels_train = output_root / 'labels' / 'train'
labels_val = output_root / 'labels' / 'val'

# Create folders
for folder in [images_train, images_val, labels_train, labels_val]:
    folder.mkdir(parents=True, exist_ok=True)

# === LOAD ANNOTATIONS ===
with open(input_root / 'data' / 'annotations.json', 'r') as f:
    coco = json.load(f)

image_id_to_info = {img['id']: img for img in coco['images']}
annotations_by_image = {}

for ann in coco['annotations']:
    image_id = ann['image_id']
    annotations_by_image.setdefault(image_id, []).append(ann)

category_map = {cat['id']: idx for idx, cat in enumerate(coco['categories'])}
category_names = [cat['name'] for cat in sorted(coco['categories'], key=lambda x: category_map[x['id']])]

# === SPLIT DATA ===
all_image_ids = list(image_id_to_info.keys())
train_ids, val_ids = train_test_split(all_image_ids, train_size=train_ratio, random_state=42)

def convert_bbox_to_yolo(bbox, img_w, img_h):
    """Convert COCO bbox to normalized YOLO format.

    Args:
        bbox (tuple): Bounding box as (x, y, w, h).
        img_w (int): Image width.
        img_h (int): Image height.

    Returns:
        tuple: Normalized (x_center, y_center, w, h).
    """
    x, y, w, h = bbox
    x_center = (x + w / 2) / img_w
    y_center = (y + h / 2) / img_h
    w /= img_w
    h /= img_h
    return x_center, y_center, w, h

def process_image(image_id, split):
    """Processes a single image and its annotations for YOLO training.

    Args:
        image_id (str): Unique identifier for the image.
        split (str): Either 'train' or 'val', indicating dataset split.

    Returns:
        None. Writes output image and label files to file system.

    """
    image_info = image_id_to_info[image_id]
    img_path = input_root / 'data' / image_info['file_name']
    img_w, img_h = image_info['width'], image_info['height']

    # Output paths
    out_img_dir = images_train if split == 'train' else images_val
    out_label_dir = labels_train if split == 'train' else labels_val

    # The TACO dataset has multiple different batches, which contain jpgs
    # that have a 6 digit name. Different pictures sometimes have the same name,
    # eg 000123.jpg in the batch_1 folder might be a different image than
    # 000123.jpg in batch_10. To ensure unique names for all images, append
    # the batch name to the image, so "000123.jpg" in the batch_1 folder
    # becomes "batch_1_000123.jpg".
    rel_path = Path(image_info['file_name'])  # e.g., batch_1/000123.jpg
    batch_name = rel_path.parts[0]
    batch_prefix = batch_name.replace('/', '_')
    filename = f"{batch_prefix}_{rel_path.name}"

    # Output flattened path
    out_img_path = out_img_dir / filename
    out_label_path = out_label_dir / filename.replace('.jpg', '.txt')

    # Copy image
    shutil.copy(input_root / 'data' / rel_path, out_img_path)

    # Write label
    with open(out_label_path, 'w') as label_file:
        for ann in annotations_by_image.get(image_id, []):
            class_id = category_map[ann['category_id']]
            bbox = convert_bbox_to_yolo(ann['bbox'], img_w, img_h)
            label_file.write(f"{class_id} {' '.join(f'{x:.6f}' for x in bbox)}\n")

# Process images
for image_id in train_ids:
    process_image(image_id, 'train')

for image_id in val_ids:
    process_image(image_id, 'val')

# === WRITE data.yaml ===
data_yaml = {
    'path': str(output_root),
    'train': 'images/train',
    'val': 'images/val',
    'nc': len(category_names),
    'names': category_names
}

with open(output_root / 'data.yaml', 'w') as f:
    yaml.dump(data_yaml, f)

print("Conversion complete. YOLO dataset created at:", output_root)

To verify that your conversion worked, make sure you can train a model and that it outputs images with a bounding box and label.

In [ ]:
!pip install -U ultralytics

from ultralytics import YOLO
import os

In [ ]:
model = YOLO('yolo11n.pt')
results = model.train(data='/kaggle/working/taco_yolo/data.yaml', epochs=5, imgsz=640)  # epoch size is small - this is just to see if it can work!

If the model outputs even one image with a bounding box and label, then the dataset should work for our project! Verify this using the code below.

In [ ]:
import cv2
from random import sample
import matplotlib.pyplot as plt

model = YOLO('runs/detect/train/weights/best.pt')

train_images_path = Path('/kaggle/working/taco_yolo/images/train')
image_files = list(train_images_path.glob('*.jpg'))

sample_images = sample(image_files, 10)

for image_path in sample_images:
    result = model(image_path)[0]
    annotated_image = result.plot()

    plt.figure(figsize=(8, 6))
    plt.imshow(annotated_image)
    plt.title(f'Predictions: {image_path.name}')
    plt.axis('off')
    plt.show()

If the model successfully generated even one image with a bounding box and label, please run the following code block to zip the taco_yolo dataset, download the zipped file, and upload it on Google Drive, https://www.google.com/url?q=https%3A%2F%2Fdrive.google.com%2Fdrive%2Ffolders%2F1bUkIYQRXX08OKI5TuOSg-eqntSudGaFB.

In [ ]:
from datetime import datetime

dataset_name = 'taco_yolo'

# Generate date prefix
date_str = datetime.now().strftime('%Y%m%d')
zip_name = f"{date_str}_{dataset_name}.zip"

# Change directory and zip
%cd /kaggle/working/{dataset_name}
!zip -r /content/{zip_name} .

print(f"Zip created at /content/{zip_name}")